In [114]:
import pandas as pd
from IPython.display import HTML, clear_output


df = pd.read_json('regen.json')
df.fillna(value={'curation_status': 0, 'curation_message': ''}, inplace=True)
df.tail(3)

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
97,Design a portfolio website for a graphic desig...,"<!DOCTYPE html>\n<html lang=""en"">\n<head>\n <...",{'Create a website for a photography portfolio...,0.395503,0.0,
98,Develop a website for a photography business t...,"<!DOCTYPE html>\n<html lang=""en"">\n<head>\n <...",{'Create a minimalist website for a photograph...,0.405975,0.0,
99,Build a website for an e-commerce store sellin...,"<!DOCTYPE html>\n<html lang=""en"">\n<head>\n <...",{'Develop a website for a photography business...,0.387743,0.0,


In [115]:
curation_df = pd.DataFrame(columns=['curation_status', 'curation_message'])

In [116]:
for idx, row in df[df['curation_status'] == 0].iterrows():
  last_idx = idx
  instruction, output, most_similar_instructions, avg_similarity_score, _, __ = row
  stop = False
  while not stop:
    clear_output(wait=True)
    display(HTML(output))
    
    print(f'''Instruction: {instruction}
    Output: {output}''')
    
    curator_input = input(f'{idx}. Is the data good? (yes/no/mark/stop):')
    curator_input = curator_input.strip()
    if curator_input == 'yes':
      curation_df.loc[idx] = {
        'curation_status': 1,
        'curation_message': '',
      }
      break
    elif curator_input == 'no':
      curation_df.loc[idx] = {
        'curation_status': -1,
        'curation_message': '',
      }
      break
    # Mark the data if you think the data can be fixed/improved manually
    elif curator_input == 'mark':
      suggestion = input('Suggestion: ')
      curation_df.loc[idx] = {
        'curation_status': 0,
        'curation_message': suggestion,
      }
      break
    elif curator_input == 'stop':
      stop = True
    else:
      print('Invalid input:', '"' + curator_input + '"')
  if stop:
    break
clear_output(wait=True)

Time,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
6:00 AM,Cardio,Yoga,Strength Training,HIIT,Cardio,Zumba,Rest
10:00 AM,Strength Training,HIIT,Cardio,Yoga,Cardio,Strength Training,Rest
4:00 PM,Yoga,Strength Training,HIIT,Zumba,Strength Training,Yoga,Rest
8:00 PM,Cardio,HIIT,Strength Training,Yoga,Zumba,Rest,Rest


Instruction: Create a landing page for a fitness center, focusing on promoting a healthy lifestyle and offering various fitness programs. The navigation bar should include links to Home, About Us, Programs, Schedule, and Contact sections. The homepage should have a hero section with an inspiring image and a captivating headline. The About Us section should provide information about the fitness center and its mission. The Programs section should highlight the different fitness programs available, showcasing their benefits and featuring images. The Schedule section should display the weekly class schedule. The Contact section should offer a contact form and essential contact details. The footer should include the fitness center's address and social media links.
    Output: <!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Fitness Center</title>
  <!-- Bootstrap 5.3 CSS -->
  <link href="https

In [111]:
curation_df

,curation_status,curation_message
20,1,
21,1,


In [112]:
curated_df = curation_df.combine_first(df)[df.columns]
curated_df.iloc[curation_df.index]

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
20,Create a clean and simple personal website tha...,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...",{'Create a modern and visually appealing portf...,0.279245,1,
21,Design a clean and minimalist website for a Di...,"<html lang=""en"">\r\n<head>\r\n <meta charset=...",{'Create a sleek and modern website for a Phot...,0.327713,1,


In [113]:
curated_df[curated_df['curation_status'] != -1].to_json('regen-curated.json', orient="records")

In [ ]:
df[['instruction', 'output']].to_json('webgen-dataset.json', orient='records')
print('webgen-dataset.json exported sucessfully!')